# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get data from `.csv` files and then preprocess data

In [2]:
df_eng = pd.read_csv("data/movie_reviews_eng.csv")

In [3]:
df_eng.sample(10)

Movie / TV Series  Year  \
5            Blade Runner 2049  2017   
1             The Dark Knight   2008   
7                The Nice Guys  2016   
0    The Shawshank Redemption   1994   
4                    Inception  2010   
2                 Forrest Gump  1994   
6  Scott Pilgrim vs. the World  2010   
9                   The Island  2005   
8      Solo: A Star Wars Story  2018   
3               The Godfather   1972   

                                            Synopsis  \
5  Officer K (Ryan Gosling), a new blade runner f...   
1  Batman (Christian Bale) teams up with District...   
7  In 1970s Los Angeles, a private eye (Ryan Gosl...   
0  Andy Dufresne (Tim Robbins), a successful bank...   
4  Dom Cobb (Leonardo DiCaprio) is a skilled thie...   
2  Forrest Gump (Tom Hanks) is a simple man with ...   
6  Scott Pilgrim (Michael Cera) must defeat his n...   
9  In a future where people are cloned for organ ...   
8  A young Han Solo (Alden Ehrenreich) joins a gr...   
3  Don Vito Corleone (Marlon Brando) is the head ...   

                                              Review  
5  "Boring and too long. Nothing like the origina...  
1  "The Dark Knight is a thrilling and intense su...  
7  "The Nice Guys tries too hard to be funny, and...  
0  "The Shawshank Redemption is an inspiring tale...  
4  "Inception is a mind-bending and visually stun...  
2  "Forrest Gump is a heartwarming and inspiratio...  
6  "It was difficult to sit through the whole thi...  
9  "The Island is a bland and forgettable sci-fi ...  
8  "Dull and pointless, with none of the magic of...  
3  "The Godfather is a classic movie that stands ...

In [4]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_eng['Original_Language'] = 'English'

    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr['Original_Language'] = 'French'

    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp['Original_Language'] = 'Spanish'

    #Ensures that all DataFrames have the same column names
    df_fr.columns = df_eng.columns
    df_sp.columns = df_eng.columns

    #Combines the three DataFrames (df_eng, df_French, and df_Spanish) into a single DataFrame named df.
    #Resets the index of the combined DataFrame to avoid issues with duplicate indices.
    df = pd.concat([df_eng, df_fr, df_sp]).reset_index(drop='True')

    df.rename(columns={'Movie / TV Series': 'Title'}, inplace=True)
    return df

df = preprocess_data()

In [5]:
df.sample(10)

Title         Year  \
20                                 Roma         2018   
8               Solo: A Star Wars Story         2018   
18            Les Visiteurs en Amérique         2000   
25                         Águila Roja   (2009-2016)   
13                        Les Choristes         2004   
4                             Inception         2010   
2                          Forrest Gump         1994   
12                               Amélie         2001   
10                           La La Land         2016   
14  Le Fabuleux Destin d'Amélie Poulain         2001   

                                             Synopsis  \
20  Cleo (Yalitza Aparicio) es una joven empleada ...   
8   A young Han Solo (Alden Ehrenreich) joins a gr...   
18  Dans cette suite de la comédie française Les V...   
25  Esta serie de televisión española sigue las av...   
13  Ce film raconte l'histoire d'un professeur de ...   
4   Dom Cobb (Leonardo DiCaprio) is a skilled thie...   
2   Forrest Gump (Tom Hanks) is a simple man with ...   
12  Cette comédie romantique raconte l'histoire d'...   
10  Cette comédie musicale raconte l'histoire d'un...   
14  Cette comédie romantique raconte l'histoire d'...   

                                               Review Original_Language  
20  "Roma es una película hermosa y conmovedora qu...           Spanish  
8   "Dull and pointless, with none of the magic of...           English  
18  "Le film est une perte de temps totale. Les bl...            French  
25  "Águila Roja es una serie aburrida y poco inte...           Spanish  
13  "Les Choristes est un film magnifique qui vous...            French  
4   "Inception is a mind-bending and visually stun...           English  
2   "Forrest Gump is a heartwarming and inspiratio...           English  
12  "Amélie est un film absolument charmant qui vo...            French  
10  "La La Land est un film absolument magnifique ...            French  
14  "Le Fabuleux Destin d'Amélie Poulain est un fi...            French

### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [6]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)

fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews =  df[df.Original_Language == 'French']['Review']
fr_reviews_en = fr_reviews.apply(lambda x: translate(x, model=fr_en_model, tokenizer=fr_en_tokenizer))

# filter synopsis in French and translate to English
fr_synopsis = df[df.Original_Language == 'French']['Synopsis']
fr_synopsis_en = fr_synopsis.apply(lambda x: translate(x, model=fr_en_model, tokenizer=fr_en_tokenizer))

# filter reviews in Spanish and translate to English
es_reviews = df[df.Original_Language == 'Spanish']['Review']
es_reviews_en = es_reviews.apply(lambda x: translate(x, model=es_en_model, tokenizer=es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df[df.Original_Language == 'Spanish']['Synopsis']
es_synopsis_en = es_synopsis.apply(lambda x: translate(x, model=es_en_model, tokenizer=es_en_tokenizer))

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data

df.loc[df.Original_Language == 'French', 'Review'] = fr_reviews_en
df.loc[df.Original_Language == 'French', 'Synopsis'] = fr_synopsis_en
df.loc[df.Original_Language == 'Spanish', 'Review'] = es_reviews_en
df.loc[df.Original_Language == 'Spanish', 'Synopsis'] = es_synopsis_en

/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
df.sample(10)

### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [54]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    result = classifier(text)[0]
    sentiment = "Positive" if result['label'] == 'POSITIVE' else "Negative"

    return sentiment

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [55]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df['Sentiment'] = df['Review'].apply(lambda x: analyze_sentiment(x, sentiment_classifier))


In [56]:
df.sample(10)

Title  Year  \
28   Torrente: El brazo tonto de la ley  1998   
12                               Amélie  2001   
2                          Forrest Gump  1994   
3                        The Godfather   1972   
29                         El Incidente  2014   
6           Scott Pilgrim vs. the World  2010   
14  Le Fabuleux Destin d'Amélie Poulain  2001   
7                         The Nice Guys  2016   
18            Les Visiteurs en Amérique  2000   
19                         Babylon A.D.  2008   

                                             Synopsis  \
28  In this Spanish comedy, a corrupt cop (played ...   
12  This romantic comedy tells the story of Amélie...   
2   Forrest Gump (Tom Hanks) is a simple man with ...   
3   Don Vito Corleone (Marlon Brando) is the head ...   
29  In this Mexican horror film, a group of people...   
6   Scott Pilgrim (Michael Cera) must defeat his n...   
14  This romantic comedy tells the story of Amélie...   
7   In 1970s Los Angeles, a private eye (Ryan Gosl...   
18  In this continuation of the French comedy The ...   
19  In the distant future, a mercenary has to esco...   

                                               Review Original_Language  \
28  "Torrente is a vulgar and offensive film that ...           Spanish   
12  "Amélie is an absolutely charming film that wi...            French   
2   "Forrest Gump is a heartwarming and inspiratio...           English   
3   "The Godfather is a classic movie that stands ...           English   
29  "The Incident is a boring and frightless film ...           Spanish   
6   "It was difficult to sit through the whole thi...           English   
14  "The Fabulous Destiny of Amélie Poulain is an ...            French   
7   "The Nice Guys tries too hard to be funny, and...           English   
18  "The film is a total waste of time. The jokes ...            French   
19  "This film is a complete mess. The characters ...            French   

   Sentiment  
28  Negative  
12  Positive  
2   Positive  
3   Positive  
29  Negative  
6   Negative  
14  Positive  
7   Negative  
18  Negative  
19  Negative

In [57]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'result'